In [1]:
# 匯入套件
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint

# 一般用法
---

In [2]:
# 取得新聞列表
url = "https://www.ptt.cc/bbs/NBA/index.html" 

# 用 requests 的 get 方法把網頁抓下來
res = req.get(url) 

# 指定 lxml 作為解析器
soup = bs(res.text, "lxml") 

# 建立 list 來放置列表資訊
list_posts = []

In [3]:
# 清空放置列表資訊的變數
list_posts.clear()

# 取得 列表 的文字與超連結
for a in soup.select('div.r-ent div.title a[href]'):
    print(a.get_text())
    print(a['href']) # 或是 a.get('href') 
    
    # 加入列表資訊
    list_posts.append({
        'title': a.get_text(),
        'link': 'https://www.ptt.cc' + a['href']
    })

[花邊] 老巴盛讚雷霆:他們可以輕易贏下西區
/bbs/NBA/M.1733903879.A.D97.html
[新聞] 點名湖人必須盡速送走1球員 美媒:可創造
/bbs/NBA/M.1733908471.A.764.html
[情報] VC: LBJ每場35分鐘太久了 該找時間休息
/bbs/NBA/M.1733910927.A.BA8.html
[討論] 麥克喬丹的生涯整體宰制力為何如此驚人
/bbs/NBA/M.1733914811.A.C97.html
[花邊] NC-公鹿隊Prince已連續3場沒得分
/bbs/NBA/M.1733917684.A.2D8.html
[公告] 板規10.1
/bbs/NBA/M.1728456762.A.CB2.html
[情報] NBA Cup Schedule (12/01 ~ 12/31)
/bbs/NBA/M.1733101092.A.1F5.html
[公告] NBA 板 開始舉辦樂透!
/bbs/NBA/M.1733892439.A.BBF.html
[情報] NBA Standings (Dec. 11, 2024)
/bbs/NBA/M.1733893089.A.B5D.html


In [ ]:
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")
    
    # 去掉 div.article-metaline (作者、標題、時間…等)
    for div in soup_.select('div[class^="article-metaline"]'):
        div.decompose()
    # decompose() 刪除標籤並銷毀，包含所有子元素，釋放資源
        
    # 去掉 div.push (推文: 推、→、噓) (判斷元素是否存在)
    if len( soup_.select('div.push') ) > 0:
        for div in soup_.select('div.push'):
            div.decompose()
    
    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML) ， 不含本身
    html = soup_.select_one('div#main-content').decode_contents()

    # 類似 JavaScript outerHTML ，含本身
    # html = str(soup_.select_one('div#main-content')) 
    
    # 整合到列表資訊的變數當中
    list_posts[index]['html'] = html


# 預覽所有結果
pprint(list_posts)

[{'html': '\n'
          '<a href="https://x.com/ClutchPoints/status/1866691236809056529" '
          'rel="noreferrer noopener nofollow" '
          'target="_blank">https://x.com/ClutchPoints/status/1866691236809056529</a>\n'
          '@ClutchPoints\n'
          '"They can go big, they can go small... This team is the clear '
          'favorite to win\n'
          'the West, easily."\n'
          '\n'
          'Charles Barkley with some HIGH praise for the Thunder\n'
          '(via@NBAonTNT)\n'
          '\n'
          '「他們可以應對大個子，也可以應對小陣容...這隊明顯是可以輕鬆贏下西區的隊伍。」\n'
          '\n'
          '老巴盛讚雷霆。\n'
          '\n'
          '\n'
          '--\n'
          '\n'
          '下面有個吐槽:\n'
          '\n'
          '@paklayLover\n'
          'Yeah thats what you said last year too\n'
          '\n'
          '對啦你去年也是這樣講的\n'
          '\n'
          '\n'
          '\n'
          '\n'
          '\n'
          '--\n'
          '<span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 118.163.112.2 (臺灣)\n

# 思考
- 如何取得**多個分頁**的內容?
  - 觀察分頁數字在網址的呈現方式
  - 將觀察到的分頁數字嵌入對應的網址當中

In [ ]:
# 清空放置列表資訊的變數
list_posts.clear()

# 起始頁數
init_page = 6503

# 最新頁數
latest_page = 6504

# 在已經知道分頁數的情況下
for page in range(init_page, latest_page + 1):
    
    # 取得新聞列表
    url = f"https://www.ptt.cc/bbs/NBA/index{page}.html" 

    # 用 requests 的 get 方法把網頁抓下來
    res = req.get(url) 

    # 指定 lxml 作為解析器
    soup = bs(res.text, "lxml") 
    
    # 取得 列表 的文字與超連結
    for a in soup.select('div.r-ent div.title a[href]'):
        # 加入列表資訊
        list_posts.append({
            'title': a.get_text(),
            'link': 'https://www.ptt.cc' + a['href']
        })
        
# 走訪每一個 a link，整合網頁內文
for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    soup_ = bs(res_.text, "lxml")

    # 去掉 div.article-metaline (作者、標題、時間…等)
    for div in soup_.select('div[class^="article-metaline"]'):
        div.decompose()
        
    # 去掉 div.push (推文: 推、→、噓) (判斷去掉元素是否存在)
    if len( soup_.select('div.push') ) > 0:
        for div in soup_.select('div.push'):
            div.decompose()

    # 取得實際需要的內容 (類似 JavaScript 的 innerHTML)
    html = soup_.select_one('div#main-content').decode_contents()

    # 整合到列表資訊的變數當中
    list_posts[index]['html'] = html

    # 可以加個 sleep 保持禮貌
        
# 預覽所有結果
pprint(list_posts)

[{'html': '\n'
          '<a href="https://x.com/nba/status/1865982953760678113?s=46" '
          'rel="noreferrer noopener nofollow" '
          'target="_blank">https://x.com/nba/status/1865982953760678113?s=46</a>\n'
          '\n'
          'BRONNY CHASEDOWN.\n'
          '\n'
          'Exclamation point on the @Lakers W!\n'
          '\n'
          '布朗尼追身大火鍋！\n'
          '\n'
          '<a href="https://i.imgur.com/b2XzGsh.gif" rel="noreferrer noopener '
          'nofollow" target="_blank">https://i.imgur.com/b2XzGsh.gif</a>\n'
          '<div class="richcontent"><img alt="" loading="lazy" '
          'src="https://cache.ptt.cc/c/https/i.imgur.com/b2XzGsl.gif?e=1734033005&amp;s=FfeUEI6alf6D--1hKRmxBg"/></div>\n'
          '<a href="https://i.imgur.com/sZlryhc.jpeg" rel="noreferrer noopener '
          'nofollow" target="_blank">https://i.imgur.com/sZlryhc.jpeg</a>\n'
          '<div class="richcontent"><img alt="" loading="lazy" '
          'src="https://cache.ptt.cc/c/https/i.